In [ ]:
%%writefile AntColonyAlgorithm.cu

#include <iostream>
#include <vector>
#include <cmath>
#include <cstdlib>
#include <ctime>
#include <limits>
#include <random>
#include <algorithm>
#include <chrono>

// CUDA
#include <cuda_runtime.h>
#include <device_launch_parameters.h>
#include <curand_kernel.h>  // <== Rastgele sayı üretmek için gerekli

using namespace std;
using namespace chrono;

// ========================================================
// Schwefel Fonksiyonu (GPU'da çalıştırılacak)
// ========================================================
__device__ double schwefelDevice2D(double x, double y)
{
    // 2 boyutlu Schwefel sabiti: 418.9829 * 2.0
    double result = 418.9829 * 2.0;
    result -= x * sin(sqrt(fabs(x)));
    result -= y * sin(sqrt(fabs(y)));
    return result;
}

// ========================================================
// GPU'da geçiş olasılıklarını hesaplama çekirdeği
// ========================================================
__global__ void calculateProbabilities(
    const double* pheromones,  // Feromon yoğunlukları (dimension=2)
    const double* heuristics,  // Heuristik bilgiler
    double* probabilities,     // Çıktı: Her karınca için 2 boyutlu olasılıklar
    int numAnts, int dimension,
    double alpha, double beta
)
{
    int antIdx = blockIdx.x * blockDim.x + threadIdx.x;
    if (antIdx >= numAnts) return;

    // 2 boyut için (dimension=2) basit bir döngü
    double numerator[2];
    double denominator = 0.0;

    for (int d = 0; d < dimension; d++) {
        double tau = pheromones[d];
        double eta = heuristics[d];
        double val = pow(tau, alpha) * pow(eta, beta);
        numerator[d] = val;
        denominator += val;
    }

    // Olasılıkları hesapla
    for (int d = 0; d < dimension; d++) {
        probabilities[antIdx * dimension + d] = numerator[d] / denominator;
    }
}

// ========================================================
// GPU'da aday çözümler ve Schwefel maliyetlerinin hesaplanması
// ========================================================
//  --> Burada curand (GPU rastgelelik) kullanıyoruz.
__global__ void generateCandidatesAndEvaluate(
    const double* probabilities,  // Karıncalar için hesaplanmış geçiş olasılıkları
    double* candidates,           // Aday çözümler (numAnts * dimension)
    double* costs,                // Schwefel maliyetleri (numAnts)
    double minCoord, double maxCoord,
    int numAnts, int dimension,
    unsigned long long seed       // Rastgele tohum
)
{
    int antIdx = blockIdx.x * blockDim.x + threadIdx.x;
    if (antIdx >= numAnts) return;

    // curand state
    curandState state;
    // Her thread farklı bir seed almalı (antIdx + blockIdx.x vs.):
    curand_init(seed, antIdx, 0, &state);

    // dimension=2 varsayımıyla:
    double xProb = probabilities[antIdx * dimension + 0];
    double yProb = probabilities[antIdx * dimension + 1];

    // 0 ile 1 arasında rastgele sayılar çekiyoruz
    double rx = curand_uniform_double(&state);
    double ry = curand_uniform_double(&state);

    // "Olasılık + rastgelelik" karışımı
    // Bu tamamen bir örnek yaklaşımdır.
    // xProb ve yProb ile tam rasgele (rx, ry) değerlerini harmanlıyoruz.
    double w = 0.5;  // katsayı, isterseniz değiştirin
    double x = minCoord + ((1.0 - w) * xProb + w * rx) * (maxCoord - minCoord);
    double y = minCoord + ((1.0 - w) * yProb + w * ry) * (maxCoord - minCoord);

    // Aday çözümü kaydet
    candidates[antIdx * dimension + 0] = x;
    candidates[antIdx * dimension + 1] = y;

    // Schwefel fonksiyon maliyeti
    costs[antIdx] = schwefelDevice2D(x, y);
}

// ========================================================
// Ana Program
// ========================================================
int main()
{
    // Süre ölçüm başlangıcı
    auto start = high_resolution_clock::now();

    // Parametreler
    const int    numAnts       = 1000;
    const int    maxIterations = 200;
    const int    dimension     = 2;
    const double desiredCost   = 1e-6; // Erken durdurma kriteri
    const double minCoord      = 400.0;
    const double maxCoord      = 450.0;

    // Feromon parametreleri
    double pheromoneEvapRate = 0.2;
    double pheromoneInit     = 1.0;
    double alpha             = 1.0;  // Feromonun etkisi
    double beta              = 2.0;  // Heuristiğin etkisi

    // CPU tarafı hazırlık
    vector<double> pheromones(dimension, pheromoneInit);
    vector<double> heuristics(dimension, 1.0 / (maxCoord - minCoord));

    vector<double> bestSolution(dimension, 0.0);
    double bestCost = numeric_limits<double>::infinity();

    // GPU bellek ayırma
    double *d_pheromones    = nullptr;
    double *d_heuristics    = nullptr;
    double *d_probabilities = nullptr;
    double *d_candidates    = nullptr;
    double *d_costs         = nullptr;

    cudaMalloc((void**)&d_pheromones,    dimension * sizeof(double));
    cudaMalloc((void**)&d_heuristics,    dimension * sizeof(double));
    cudaMalloc((void**)&d_probabilities, numAnts   * dimension * sizeof(double));
    cudaMalloc((void**)&d_candidates,    numAnts   * dimension * sizeof(double));
    cudaMalloc((void**)&d_costs,         numAnts   * sizeof(double));

    // Başlangıç verilerini GPU'ya kopyala
    cudaMemcpy(d_pheromones, pheromones.data(), dimension * sizeof(double), cudaMemcpyHostToDevice);
    cudaMemcpy(d_heuristics, heuristics.data(), dimension * sizeof(double), cudaMemcpyHostToDevice);

    // Paralel döngü konfigürasyonu
    int threadsPerBlock = 128;
    int blocks          = (numAnts + threadsPerBlock - 1) / threadsPerBlock;

    // Rastgele tohum (isteğe göre değiştirebilirsiniz)
    unsigned long long seed = 1234ULL;

    for (int iter = 0; iter < maxIterations; iter++)
    {
        // 1) Olasılıkları hesapla
        calculateProbabilities<<<blocks, threadsPerBlock>>>(
            d_pheromones,
            d_heuristics,
            d_probabilities,
            numAnts,
            dimension,
            alpha,
            beta
        );
        cudaDeviceSynchronize();

        // 2) Adayları oluştur + Schwefel maliyetlerini hesapla (gpu)
        generateCandidatesAndEvaluate<<<blocks, threadsPerBlock>>>(
            d_probabilities,
            d_candidates,
            d_costs,
            minCoord,
            maxCoord,
            numAnts,
            dimension,
            seed
        );
        cudaDeviceSynchronize();

        // 3) CPU'ya kopyala, en iyi adayı bul
        vector<double> costs(numAnts);
        vector<double> candidates(numAnts * dimension);

        cudaMemcpy(costs.data(),      d_costs,      numAnts * sizeof(double),             cudaMemcpyDeviceToHost);
        cudaMemcpy(candidates.data(), d_candidates, numAnts * dimension * sizeof(double), cudaMemcpyDeviceToHost);

        double iterationBestCost = numeric_limits<double>::infinity();
        vector<double> iterationBestSolution(dimension, 0.0);

        for (int ant = 0; ant < numAnts; ant++)
        {
            if (costs[ant] < iterationBestCost) {
                iterationBestCost        = costs[ant];
                iterationBestSolution[0] = candidates[ant * dimension + 0];
                iterationBestSolution[1] = candidates[ant * dimension + 1];
            }
        }

        // 4) Global en iyiyi güncelle
        if (iterationBestCost < bestCost)
        {
            bestCost     = iterationBestCost;
            bestSolution = iterationBestSolution;
        }

        // 5) Feromon güncellemesi (basit)
        for (int d = 0; d < dimension; d++) {
            // Buharlaşma
            pheromones[d] *= (1.0 - pheromoneEvapRate);
            // Iterasyon en iyisi bazlı ekleme
            pheromones[d] += 1.0 / (1.0 + iterationBestCost);
        }

        // GPU'ya güncel feromonları kopyala
        cudaMemcpy(d_pheromones, pheromones.data(), dimension * sizeof(double), cudaMemcpyHostToDevice);

        // Aralıklarla sonuç raporu
        if (iter % 100 == 0)
        {
            cout << "[Iter " << iter << "] "
                 << "Best Cost so far = " << bestCost << endl;
        }

        // 6) Erken durdurma
        if (bestCost < desiredCost) {
            cout << "\nErken durdurma: hedef cost (" << desiredCost << ") sağlandı.\n";
            break;
        }
    }

    // GPU bellek temizleme
    cudaFree(d_pheromones);
    cudaFree(d_heuristics);
    cudaFree(d_probabilities);
    cudaFree(d_candidates);
    cudaFree(d_costs);

    // Süre ölçüm sonu
    auto end = high_resolution_clock::now();
    duration<double> elapsedTime = end - start;

    // Sonuçları yazdır
    cout << "\n** En iyi sonuç (cost): " << bestCost << "\n";
    cout << "** En iyi (x, y): ";
    for (auto val : bestSolution) {
        cout << val << " ";
    }
    cout << "\nAlgoritmanın çalışması " << elapsedTime.count() << " saniye sürdü.\n";

    return 0;
}

Overwriting AntColonyAlgorithm.cu


In [ ]:
!nvcc -o AntColonyAlgorithm AntColonyAlgorithm.cu
!./AntColonyAlgorithm

[Iter 0] Best Cost so far = 0.00141577
[Iter 100] Best Cost so far = 0.00141577
[Iter 200] Best Cost so far = 0.00141577
[Iter 300] Best Cost so far = 0.00141577
[Iter 400] Best Cost so far = 0.00141577
[Iter 500] Best Cost so far = 0.00141577
[Iter 600] Best Cost so far = 0.00141577
[Iter 700] Best Cost so far = 0.00141577
[Iter 800] Best Cost so far = 0.00141577
[Iter 900] Best Cost so far = 0.00141577

** En iyi sonuç (cost): 0.00141577
** En iyi (x, y): 420.865 420.982 
Algoritmanın çalışması 0.477164 saniye sürdü.
